## Sketchbook

Simple sketchbook where we run functions that we will modify in the main code (tfbs.ipynb) to check and see if it does what we expect it to.

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import math as m
from numpy.polynomial.legendre import leggauss
from scipy.special import sph_harm
from sympy.physics.quantum.cg import CG, Wigner9j

Some functions we need to test the permutation operator.

Some things we need to note:
- $p, q$ are equivalent to $p_{12}, p_3$ respectively.
- Here $p_3^\prime$ is only in the z-direction, so the magnetic component is zero (orbital ang mom only
in one direction), i.e. $m_{3} = 0 \implies m_{12}=M$. But $p_3$ is in both x and z direction and so can vary (defined via $\mu$, $M-\mu$ for $m_{12}, m_3$ resp.) 

Below is the transformation method to transform the grid points for the Gauss-Legendre quadrature (needed to get grid and weights for integration)

In [14]:
def _trns(np1,np2,pa,pb,pc):
  """Auxilliary method that provides transformed Gauss-Legendre grid points and integration weights.

     This is using a hyperbolic trafo shown in the lecture. 
     Parameter: 
     np1 --  grid points in ]0,pb[
     np2 --  grid points are distributed in ]pb,pc[ using a linear trafo

     pa  -- half of np1 points are in interval [0,pa]
     pb  -- interval boundary as defined above 
     pc  -- upper integration boundary """ 

  x1grid,x1weight=leggauss(np1)
  x2grid,x2weight=leggauss(np2)

  npoints = np1 + np2

  # trafo (1.+X) / (1./P1-(1./P1-2./P2)*X) for first interval 
  p1grid=(1.+x1grid) / (1./pa-(1./pa-2./pb)*x1grid)
  p1weight=(2.0/pa-2.0/pb)*x1weight / (1./pa-(1./pa-2./pb)*x1grid)**2

  # linear trafo 
  p2grid=(pc+pb)/2.0 + (pc-pb)/2.0*x2grid
  p2weight=(pc-pb)/2.0*x2weight

  pgrid=np.empty((npoints),dtype=np.double)
  pweight=np.empty((npoints),dtype=np.double)

  pgrid = np.concatenate((p1grid, p2grid), axis=None)
  pweight = np.concatenate((p1weight, p2weight), axis=None)

  return pgrid,pweight 

Below is auxiliary function that converts three component vector into spherical coordinates of $(p, \theta, \phi)$ (ex. $\vec{p_{12}} = (p_{12}, \theta_{12}, \phi_{12})$)

In [15]:
def _angle(px,py,pz):
    """Auxiliary routine to determine magnitude, phi, and theta of three component vector. 

       Parameters:
       px,py,pz -- cartesian components of a vector 

       returns magntitude, theta and phi angles.
    """

    pmag=np.sqrt(px**2+py**2+pz**2)
    theta=np.where(pmag!=0.0,np.arccos(pz/pmag),0.0)

    phi=theta # copy shape of theta to phi 
    phi=1.5*m.pi # and set to constant value

    # prepare bool arrays for px,py > 0 < 0  with shape of phi 

    pxgt0=(phi==0)  # copy shape
    pxgt0=(px>0)    # test 

    pxlt0=(phi==0)  # copy shape
    pxlt0=(px<0)    # test 

    pxeq0=(phi==0)  # copy shape
    pxeq0=(px==0)   # test 

    pygt0=(phi==0)  # copy shape
    pygt0=(py>0)    # test 

    np.where(pxgt0 & pygt0,np.arctan(py/px),phi)
    np.where(pxgt0 & np.invert(pxgt0),2*m.pi-np.arctan(-py/px),phi)
    np.where(pxlt0 & pygt0,m.pi-np.arctan(-py/px),phi)
    np.where(pxlt0 & np.invert(pygt0),m.pi+np.arctan(py/px),phi)
    np.where(pxeq0 & pygt0,0.5*m.pi,phi)

    return pmag,theta,phi   

below is auxiliary function for converting index $l, m$ into a combined index.

In [18]:
def _lmindx(l,m):
    """Combined unique index for l and m.
    
        Nice trick: since quantum numbers lm are linked to each other, this combined 
        index allows one to store the results depending on lm using the memory more efficiently. 
    """        
    return l**2+l+m

Here below we test the execution of the permutation operator.

In [22]:
# some initial parameters set by ThreeBody (inherited from TwoBodyTMat) class

# gridpoints for momentum transfer
nq1=20
nq2=10
nqpoints = nq1 + nq2

# grid points for x (cos of angle between p3 and p3')
nx = 12

lmax=0  # max two-body angular momentum
bl=0  # total orbital angular momentum

# momentum grid interval boundaries
qa=1.0
qb=5.0
qc=20.0

xp,xw = leggauss(nx)  # grid points for permutation op

bl=bl  # total angular momentum
lammax=lmax+bl  # max value for l3
qnalpha=[]
alpha=0

# creating dictionaries for alpha, that contains l12, l3, L
for l in range(lmax+1):
  if(l%2==0):   # take only symmetric pw (Pauli)  
   for lam in range(abs(l-bl),l+bl+1):
     qnalpha.append({"alpha":alpha,"l":l,"lam":lam,"bl":bl})
     alpha+=1
nalpha=len(qnalpha)

# creating momentum transfer grid
qgrid,qweight = _trns(nq1,nq2,qa,qb,qc)

Below is the formulation of the permutation operator.

- `pip, pi` are $\pi_{12}^\prime$, $\pi_{12}$ respectively.
- `thetapp` is the arccosine of $x$ (angle between p3 and p3')
- `theta, thetap` are the direction of p12 and p3 resp.

In [17]:
# prepare shifted momenta and angles for the symmetric permutation 

pip=np.empty((nqpoints, nqpoints, nx),dtype=np.double)        
pi=np.empty(( nqpoints, nqpoints, nx),dtype=np.double)        

thetap=np.empty(( nqpoints, nqpoints, nx),dtype=np.double)
theta=np.empty(( nqpoints, nqpoints, nx),dtype=np.double)
thetapp=np.empty(( nx),dtype=np.double)

# construct direction and magnitude vectors for p12, p3, x
for ix in range( nx):
  xval= xp[ix] 
  thetapp[ix]=np.arccos(xval)
  for jq in range( nqpoints):
    qpval= qgrid[jq]
    for iq in range( nqpoints):
      qval= qgrid[iq]

      px=qpval*np.sqrt(1.0-xval**2)
      py=0.0
      pz=0.5*qval+qpval*xval 
      pi[iq,jq,ix],theta[iq,jq,ix],phi= _angle(px,py,pz)

      px=-0.5*qpval*np.sqrt(1.0-xval**2)
      py=0.0
      pz=-qval-0.5*qpval*xval 
      pip[iq,jq,ix],thetap[iq,jq,ix],phi= _angle(px,py,pz)



Here the spherical harmonics are constructed.
Since we write the integral of the permutation operator only with respect to $x = \hat{p_3}$, the spherical harmonics are written in this fashion as well. (Lecture 9, page 6, line 3). This is contained within the definition of the permutation operator $G_{\alpha^\prime, \alpha}$

So here we find the spherical harmonics $y_{l,\mu}(\hat{-q-0.5q*p}), y_{l,\mu}(\hat{q+0.5q*p})$ and $y_{\lambda, \mu}(\hat{qp})$, where $(l, \mu) = (l_{12}, m)$ and $(\lambda, \mu) = (l_3, m)$. 

In [19]:
# prepare spherical harmonics and store based on lmindx 
# number of lam,mu und l,mu combinations 
nlamindx=_lmindx(lammax,lammax)+1
nlindx=_lmindx(lmax,lmax)+1

# array for Y_{lam mu}(hat qp) (real is sufficient since phi=0)
ylam=np.empty((nlamindx,nx),dtype=np.double)
for lam in range(lammax+1):
    for mu in range(-lam,lam+1):
        ylam[_lmindx(lam,mu),:]=np.real(sph_harm(mu,lam, 0, thetapp))


# array for Y_{l mu}(-q-0.5qp) (real is sufficient since phi=0)
yl=np.empty((nlindx,nqpoints,nqpoints,nx),dtype=np.double)
for l in range(lmax+1):
    for mu in range(-l,l+1):
        yl[_lmindx(l,mu),:,:,:]=np.real(sph_harm(mu,l, 0, thetap))

# array for Y*_{l mu}(0.5q+qp) (real is sufficient since phi=0)
ystarl=np.empty((nlindx,nqpoints,nqpoints,nx),dtype=np.double)
for l in range(lmax+1):
    for mu in range(-l,l+1):
        ystarl[_lmindx(l,mu),:,:,:]=np.real(sph_harm(mu,l, 0, theta))

Here then we evaluate the CG coefficients contained in the coupled spherical harmonics.

The primed coordinates have $(M, 0, M)$ due to how $p_3$ is defined.
The unprimed coordinates have a x-dependence so its $(\mu, M-\mu, M)$.
They also share the same total magnetic quantum number and the orbital angular momentum too (why same l12, l3?).


In [23]:
# now prepare the necessary Clebsch-Gordan coefficients
# we need (l lam L, M 0 M)  and (l lam L,mu M-mu,M)
# I assume that L is smaller than the lmax or lammax therefore M=-L,L
# the smallest index for storage 

cg=np.zeros((nalpha,2*bl+1),dtype=np.double)
cgp=np.zeros((nalpha,2*lmax+1,2*bl+1),dtype=np.double)

for qnset in qnalpha:  # go through allowed l,lam combinations
    for bm in range(-bl,bl+1):
        cg[qnset["alpha"],bm+bl]=float(CG(qnset["l"],bm,qnset["lam"],0,bl,bm).doit())
    for mu in range(-qnset["l"],qnset["l"]+1):
        cgp[qnset["alpha"],mu+qnset["l"],bm+bl]=float(CG(qnset["l"],mu,qnset["lam"],bm-mu,bl,bm).doit())

Below evaluated the coupled spherical harmonic (evaluates summation of m12 and m3)

In [31]:
# now we can perform the mu summation for the combination of coupled spherical harmonics 
ylylam=np.zeros((nalpha,2*bl+1,nqpoints,nqpoints,nx),dtype=np.double)  # coupled sph harmonic array
for qnset in qnalpha:  # go through allowed l,lam combinations
    alphap=qnset["alpha"]
    l=qnset["l"]
    lam=qnset["lam"]
    for bm in range(-bl,bl+1):
        for mu in range(-l,l+1):
            lmindx=_lmindx(l,mu)
            if abs(bm-mu)<=lam:
                lamindx=_lmindx(lam,bm-mu)
                ylylam[alphap,bm+bl,:,:,:]+=cgp[alphap,mu+l,bm+bl]*yl[lmindx,:,:,:]*ylam[lamindx,:]

Below evaluates the numerical representation of the permutation operator $G_{\alpha^\prime, \alpha}$.
So over here the summation over $M_L$ gives this.

The normalization comes from:
- summation over $M_L$
- the cancellation of the normalization coefficient from the definition of the spherical harmonics in `scipy.special.sph_harm`.

**Modification**
Here we need to add the following:
- the spin components
- the Wigner-9j coefficients

In [32]:
# bm summation then gives G 
gfunc=np.zeros((nalpha,nalpha,nqpoints,nqpoints,nx),dtype=np.double)
for qnset in qnalpha:  # go through allowed l,lam combinations
    alpha=qnset["alpha"]
    l=qnset["l"]
    lam=qnset["lam"]
    for qnsetp in qnalpha:  # go through allowed l,lam combinations
        alphap=qnsetp["alpha"]
        for bm in range(-bl,bl+1):
            if(abs(bm)<=l):  
                lmindx=_lmindx(l,bm) 
                gfunc[alpha,alphap,:,:,:]+=8*m.pi**2*np.sqrt((2*lam+1)/(4*m.pi))/(2*bl+1) \
                    *ystarl[lmindx,:,:,:]*ylylam[alphap,bm+bl,:,:,:]